#**Encephalon Diseases Classifier**
Is a multi-level project where more than one step of classification is performed on a dataset that contains MRI scans of control subjets, Alzheimer's disease, Parkinson's disease, and brain tumors.

---

##**Encephalon Dataset**
The dataset used in this project is a combintion of three different datasets. These datasets are the Augmented Alzheimer MRI Dataset, which is an augmented version of the Alzheimer’s Dataset (4 class of Images), the Brain Tumor MRI Dataset, and the NTUA Parkinson Dataset.

***The following URL is for accessing and using the dataset:*** Encephalon_Dataset, https://drive.google.com/drive/folders/16bqDxOEimwF4ASa_e52PIC48XQX6oNAp?usp=sharing.

*If you could not access the dataset through the link, feel free to e-mail us on the following e-mail addresses for help:*


1.   ladyaouto@gmail.com
2.   leen.aouto@gmail.com
3.   rawan.flifel66@gmail.com

---



## **Step 1:** Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## **Step 2:** Importing dependencies

Importing all the necessary libraries including Keras, Tensorflow, NumPy, Matplotlib, Sklearn, and more.

In [ ]:
import keras
import tensorflow
import os
import operator
import time
import itertools
import sys
import sklearn
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Add, Dense,GlobalAvgPool2D, Concatenate, AvgPool2D, Dropout, ReLU, Activation, MaxPool2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from PIL import Image
%matplotlib inline

## **Step 3:** Data preprocessing

**(a)** Augmenting data using *ImageDataGenerator* class and reading data using *flow_from_directory* mathod.

In [ ]:
train_path = '/content/drive/MyDrive/Brain_Diseases_Classification/Datasets/Encephalon_Dataset/Training/Level3/Tumor'
test_path = '/content/drive/MyDrive/Brain_Diseases_Classification/Datasets/Encephalon_Dataset/Testing/Level3/Tumor'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

print("Training set:")
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

print("---------------------------")

print("Testing set:")
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'categorical',
                                            shuffle=False)

**(b)** Printing count of each class in the target path of training and testing sets.

In [ ]:
total_train = 0
total_test = 0

print("Training Count:")
for c in os.listdir(train_path):
    print("class ({}) has {} images.".format(c, len(os.listdir(os.path.join(train_path, c)))))
    total_train += len(os.listdir(os.path.join(train_path, c)))
print("---------------------------")

print("Testing Count:")
for c in os.listdir(test_path):
    print("class ({}) has {} images.".format(c, len(os.listdir(os.path.join(test_path, c)))))
    total_test += len(os.listdir(os.path.join(test_path, c)))
print("---------------------------")

print("Total Training Images:", total_train)
print("Total Testing Images:", total_test)

**(c)** Visualizing data count of the training set.

In [ ]:
LABELS = ['Glioma', 'Meningioma', 'Pituitary']
nimgs={}
for i in LABELS:
  nimages=len(os.listdir(train_path+'/'+i))
  nimgs[i]=nimages
plt.figure(figsize=(6, 3))
plt.bar(range(len(nimgs)), list(nimgs.values()), align='center')
plt.xticks(range(len(nimgs)), list(nimgs.keys()))
plt.title('Distribution of different classes in Training Dataset')
plt.show()

**(d)** Displaying data samples.

In [ ]:
def display_sample_images(folder_path):
    for subfolder in os.listdir(folder_path):
        folder_images = []
        subfolder_path = os.path.join(folder_path, subfolder)

        for image_file in os.listdir(subfolder_path):
            if image_file.endswith(('.jpg', '.png')):
                folder_images.append(os.path.join(subfolder_path, image_file))

        plt.figure(figsize=(10, 3))
        plt.suptitle(f"Sample Images of {subfolder}", fontsize=18)
        plt.axis('off')

        num_images_to_display = min(4, len(folder_images))
        random_images = random.sample(folder_images, num_images_to_display)

        for i, image_path in enumerate(random_images, start=1):
            plt.subplot(1, num_images_to_display, i)
            img = Image.open(image_path)
            img = img.convert("RGB")
            plt.imshow(img)
            plt.axis('off')

        plt.show()


display_sample_images(train_path)

## **Step 4:** Implementating DenseNet121





**(a)** Downloading the model, then printing a summary and a visual architecture of it.

In [ ]:
DenseN = tf.keras.applications.densenet.DenseNet121(include_top=False,
    weights="imagenet",
    input_tensor=Input(shape=(224, 224, 3))
    )
keras.utils.plot_model(DenseN, "/content/drive/MyDrive/Colab Notebooks/DenseNet121_Standard.png")

**(b)** Adding customized output layers.




In [ ]:
# Freeze the layers
for layer in DenseN.layers[:-9]:
    layer.trainable= False

x= DenseN.layers[-3].output
x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(1024,activation='relu')(x)
x= Dense(512,activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)

predictions= Dense(3, activation = 'softmax')(x)
model= Model(inputs= DenseN.input, outputs= predictions)

**(c)** Defining the metrics array.

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR')
]

**(d)** Compiling the model.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer= 'adam' ,metrics=[METRICS])

**(e)** Printing model summary and visual architecture after the added output layes.

In [ ]:
model.summary()
keras.utils.plot_model(model, "/content/drive/MyDrive/Colab Notebooks/Model_Plots/DenseNet121_Customized.png")

# **Step 5**: Training and testing the model

In [ ]:
checkpoint = ModelCheckpoint('Densenet121_checkpoint_S3.h5', monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto')

early = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=20,
                      verbose=1,
                      mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                           patience=5, mode='min', verbose=1,
                           min_lr=1e-5)

Densenet121_classifier = model.fit(
  training_set,
  validation_data= test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  verbose = True,
  callbacks=[checkpoint, early, reduce_lr]
  )

# **Step 6:** Saving the model

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/Model/Densenet121.h5')
print("Model was saved.")

# **Step 7:** Evaluating Results




**(a)** Plotting Loss and Accuracy

In [ ]:
plt.figure(figsize=(12, 8))

# Loss
plt.subplot(2, 2, 1)
plt.plot(Densenet121_classifier.history['loss'], label='Loss')
plt.plot(Densenet121_classifier.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

# Accuracy
plt.subplot(2, 2, 2)
plt.plot(Densenet121_classifier.history['accuracy'], label='Accuracy')
plt.plot(Densenet121_classifier.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
# Accuracy
acc = Densenet121_classifier.history['accuracy']
epochs = range(1, len(acc) + 1)
val_acc = Densenet121_classifier.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.grid()

plt.savefig('acc.png')
plt.figure(figsize=(12, 8))
plt.show()

# Loss
loss = Densenet121_classifier.history['loss']
val_loss = Densenet121_classifier.history['val_loss']
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc='upper right')
plt.grid()

plt.savefig('loss.png')
plt.figure(figsize=(12, 8))
plt.show()

**(b)** Visualizing more metrics.

In [ ]:
def plot_metrics(history):
  metrics =  ['loss', 'auc', 'precision', 'recall', 'accuracy']
  mpl.rcParams['figure.figsize'] = (12, 10)
  colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(3,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    elif metric == 'accuracy':
      plt.ylim([0,1])
    else:
      plt.ylim([0,1])

    plt.legend()


plot_metrics(Densenet121_classifier)

**(c)** Plotting Confusion Matrix.

In [ ]:
val_accuracy = model.evaluate(test_set)

Getting true labels and predicted labels.

In [ ]:
predictions = model.predict(test_set, verbose=1)
predicted_labels = np.argmax(predictions, axis=1)

true_labels = test_set.classes

Calculating and printing the confusion matrix and the classification report.

In [ ]:
confusion_mat = confusion_matrix(true_labels, predicted_labels)
print(confusion_mat)

print("\n-----------------------------------------------------------\n")

target_names = ['Glioma', 'Meningioma', 'Pituitary']
print('Classification Report')
print(classification_report(true_labels, predicted_labels, target_names=target_names))

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(9, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()


plot_confusion_matrix(confusion_mat, target_names=target_names, title="confusion matrix")